In [2]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/'My Drive'/'Projects'/'genetics'
! ls
! pip install pyeasyga

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Projects/genetics
cifar_net.py  genetic_net_generator.ipynb  __pycache__
data	      genetics.py		   README.md
  Created wheel for pyeasyga: filename=pyeasyga-0.3.1-py2.py3-none-any.whl size=6774 sha256=11337e75bb485cd502c68190293b0b763e8d97cd442214611aeface84c57e2af
  Stored in directory: /root/.cache/pip/wheels/ef/cf/ef/7aff9fcd6c1e59dc276182f29a32e7c197665dd5eb547f30e6
Successfully built pyeasyga


In [1]:
#################################################################
# DATASET LOADING
#################################################################
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

#load and prepare dataset
#can use standard python packages to load data as a numpy array, then
#convert the array to a torch.*Tensor

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [0]:
#################################################################
#IMAGE SHOWER
#################################################################
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [2]:
#################################################################
#GENETIC ALGORITHM
#################################################################
from genetics import GeneticFit
from cifar_net import DefaultNet, train, computePerformance
genetics.reload(GeneticFit)
cifar_net.reload(DefaultNet)

"""
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
"""

validationloader = testloader

genetic = GeneticFit(testloader, validationloader, num_epochs = 3, max_iter = 5, pop_size = 3, num_generations = 2)

default = DefaultNet()
data = [default]
net = genetic.create_individual(data)
genetic.mutate(net)




input()
genetic.geneticFit(pretrained = False)


layer dimensions are [80, 100]
new layers is  [750, 80, 100, 10]
index is  1


NameError: name 'layers' is not defined